In [ ]:
! pip install llama-index

In [38]:
import nest_asyncio

nest_asyncio.apply()

import os
import keys

from llama_index import ServiceContext
from llama_index.llms import AzureOpenAI
from llama_index.schema import MetadataMode

from llama_index.embeddings import OpenAIEmbedding

In [39]:
llm = AzureOpenAI(
    engine="raidGPT",
    model="gpt-4",
    temperature=0.0,
    api_base="https://raid-ses-openai.openai.azure.com/",
    api_key=keys.gpt_key,
    api_type="azure",
    api_version="2023-05-15"
)

emb_llm = OpenAIEmbedding(
    engine="swiftfaq-ada002",
    model="text-embedding-ada-002",
    temperature=0.0,
    api_base="https://raid-ses-openai.openai.azure.com/",
    api_key=keys.gpt_key,
    api_type="azure",
    api_version="2023-05-15"
)

In [6]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
    MetadataFeatureExtractor,
)
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)


class CustomExtractor(MetadataFeatureExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": node.metadata["document_title"]
                + "\n"
                + node.metadata["excerpt_keywords"]
            }
            for node in nodes
        ]
        return metadata_list


metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=5, llm=llm),
        # EntityExtractor(prediction_threshold=0.5),
        SummaryExtractor(summaries=["prev", "self"], llm=llm),
        # KeywordExtractor(keywords=10, llm=llm),
        # CustomExtractor()
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

In [10]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('../data/124').load_data()

In [14]:
nodes = node_parser.get_nodes_from_documents(documents[10:30])

Extracting summaries: 100%|██████████| 31/31 [05:23<00:00, 10.45s/it]


In [15]:
len(nodes)

31

In [20]:
nodes[2].metadata

{'page_label': '12',
 'file_name': 'AP3456 Vol 12 Helicopters.pdf',
 'document_title': 'Understanding the Principles and Techniques of Helicopter Movement and Hovering in Various Environments',
 'prev_section_summary': 'The section is from a document titled "Understanding the Principles and Techniques of Helicopter Movement and Hovering in Various Environments". It includes various figures explaining different aspects of helicopter movement and hovering. Key topics include the effect of long grass on recirculation, recirculation near a building, producing horizontal movement, flapping to equality, control orbit, pitch operating arm movement, relationship of blade position to control orbit position, high and low blade positions, advance angle, dragging hinge, variation in radius of blade CG resulting from flapping, and Hooke’s Joint Effect.',
 'section_summary': "The section discusses the principles of helicopter hovering and horizontal movement, specifically focusing on the take-off an

In [23]:
nodes

[TextNode(id_='9fa2bf47-b15e-4007-b4b6-8ba5d198397c', embedding=None, metadata={'page_label': '11', 'file_name': 'AP3456 Vol 12 Helicopters.pdf', 'document_title': 'Understanding the Principles and Techniques of Helicopter Movement and Hovering in Various Environments', 'section_summary': 'The section is from the document titled "Understanding the Principles and Techniques of Helicopter Movement and Hovering in Various Environments" from the file "AP3456 Vol 12 Helicopters.pdf". It is the second chapter, focusing on the principles of hovering and horizontal movement in helicopters. Key topics discussed include take-off and climb to a free air hover, vertical descent, ground effect, recirculation, cyclic pitch changes, flapping to equality, control orbit, pitch operating arm movement, phase lag, advance angle, and dragging. The section also contains several figures illustrating these concepts.'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelatio

In [41]:
from llama_index import VectorStoreIndex
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

index = VectorStoreIndex(nodes, service_context=ServiceContext.from_defaults(llm=llm, embed_model=emb_llm))

In [46]:
index

In [49]:
ans = index.as_query_engine().query("What is the article talking about?")

In [51]:
print(ans.get_formatted_sources())

> Source (Doc id: 9fa2bf47-b15e-4007-b4b6-8ba5d198397c): AP3456 – 12-2 - Hovering and Horizontal Movement 
CHAPTER 2 - HOVERING AND HORIZONTAL MOVEMENT 
 ...

> Source (Doc id: 1954e9a1-fed9-4990-afb2-3985d37b194b): AP3456 – 12-2 - Hovering and Horizontal Movement 
HOVERING 
 
Take-off and Climb to a Free Air Ho...


In [52]:
print(ans)

The article discusses the principles of helicopter movement, specifically focusing on hovering and horizontal movement. It explains the process of take-off and climb to a free air hover, detailing how a force greater than the weight of the helicopter must be produced to lift it off the ground. This force acts vertically downwards through the aircraft's center of gravity. The article also discusses how the Total Rotor Thrust (TRT) and the All Up Weight (AUW) of the helicopter interact during this process. Additionally, it covers the effect of the Rate of Climb (ROC) on the induced airflow, angle of attack, and total rotor thrust.


In [53]:
index.as_retriever().retrieve("What is this article about?")

[NodeWithScore(node=TextNode(id_='9fa2bf47-b15e-4007-b4b6-8ba5d198397c', embedding=None, metadata={'page_label': '11', 'file_name': 'AP3456 Vol 12 Helicopters.pdf', 'document_title': 'Understanding the Principles and Techniques of Helicopter Movement and Hovering in Various Environments', 'section_summary': 'The section is from the document titled "Understanding the Principles and Techniques of Helicopter Movement and Hovering in Various Environments" from the file "AP3456 Vol 12 Helicopters.pdf". It is the second chapter, focusing on the principles of hovering and horizontal movement in helicopters. Key topics discussed include take-off and climb to a free air hover, vertical descent, ground effect, recirculation, cyclic pitch changes, flapping to equality, control orbit, pitch operating arm movement, phase lag, advance angle, and dragging. The section also contains several figures illustrating these concepts.'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relation